In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split,KFold
import lightgbm as lgbm

train=pd.read_csv("./data/zhengqi_train.txt",sep='\t')# V0-V37, target
test=pd.read_csv("./data/zhengqi_test.txt",sep='\t')

X=np.array(train.drop(['target'], axis = 1))#(2888, 38)
y=np.array(train['target'])#(2888,)

In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 训练集和验证集切割训练

In [31]:
lgb=LGBMRegressor()
lgb.fit(X_train,y_train)
y_pred_lgb=lgb.predict(X_test)
print(mean_squared_error(y_test,y_pred_lgb))

0.11877436844015696


# 5折交叉验证

In [33]:
folds=5

best_score=10
best_model=''

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgb=lgbm.LGBMRegressor()
    lgb.fit(X_train[train_index],y_train[train_index])
    y_pred_lgb=lgb.predict(X_train[test_index])
    score=mean_squared_error(y_pred_lgb,y_train[test_index])
    print(i,'th fold：',score)
    
    if score<best_score:
        best_score=score
        best_model=lgb

0 th fold： 0.11373521701915587
1 th fold： 0.11510770978164474
2 th fold： 0.10923676670851053
3 th fold： 0.11617842172445779
4 th fold： 0.12505113289137243


In [34]:
y_pred=best_model.predict(X_test)
print(mean_squared_error(y_pred,y_test))

0.12035009734616724


# 调参
https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMModel.html#lightgbm.LGBMModel

In [3]:
folds=5

avg_score=0

kf=KFold(n_splits=folds,shuffle=True,random_state=2022)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgb=LGBMRegressor()
    lgb.fit(X_train[train_index],y_train[train_index])
    y_pred_lgb=lgb.predict(X_train[test_index])
    score=mean_squared_error(y_pred_lgb,y_train[test_index])
    print(i,'th fold：',score)
    
    avg_score+=score
avg_score/=folds
print(avg_score)

0 th fold： 0.11373521701915587
1 th fold： 0.11510770978164474
2 th fold： 0.10923676670851053
3 th fold： 0.11617842172445779
4 th fold： 0.12505113289137243


In [31]:
import lightgbm as lgb

params = {
    'boosting_type': 'gbdt', 
    'objective': 'regression', 

    'learning_rate': 0.05, 
    'num_leaves': 50, 
    'max_depth': 6,

    'subsample': 0.8, 
    'colsample_bytree': 0.8, 
    }
data_train = lgb.Dataset(X_train, y_train, silent=True)
cv_results = lgb.cv(
    params, data_train, num_boost_round=1000, nfold=5, stratified=False, shuffle=True, metrics='mse',
    early_stopping_rounds=50, verbose_eval=50, show_stdv=True, seed=0)

print('best n_estimators:', len(cv_results['l2-mean']))
print('best cv score:', cv_results['l2-mean'][-1])

C:\Users\nanfe\anaconda3\lib\site-packages\lightgbm\engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\nanfe\anaconda3\lib\site-packages\lightgbm\basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\nanfe\anaconda3\lib\site-packages\lightgbm\engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a fut

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000818 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8846
[LightGBM] [Info] Number of data points in the train set: 1848, number of used features: 38
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000952 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8846
[LightGBM] [Info] Number of data points in the train set: 1848, number of used features: 38
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000747 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8846
[LightGBM] [Info] Number of data points in the train set: 1848, number of used features: 38
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000809 seconds.
You can set `force_c

In [ ]:
学习率0.1
best n_estimators: 275
best cv score: 0.11385408762155265

学习率0.05
best n_estimators: 437
best cv score: 0.11039529412190559
    
学习率0.01  
best n_estimators: 1000
best cv score: 0.11049402702546904

学习率0.001
best n_estimators: 1000
best cv score: 0.2493478416675996

In [18]:
cv_results

{'l2-mean': [0.7988256772746004,
  0.6776268007701285,
  0.5773298476424196,
  0.49456169425895274,
  0.42731880283752366,
  0.3729611081242603,
  0.3285093737260629,
  0.29193539339499985,
  0.26245688535754014,
  0.23771415558232242,
  0.218636688460853,
  0.20225707647356783,
  0.1886267495106596,
  0.17746210791308142,
  0.1680618652979866,
  0.16061931554440764,
  0.1547800577482195,
  0.1498498986851562,
  0.1452945683419436,
  0.1418347295414721,
  0.13878056857974125,
  0.1363963956426429,
  0.13431647945336173,
  0.13257801741861405,
  0.1310400593997192,
  0.12953417053267177,
  0.12825323967929747,
  0.12708174360270222,
  0.12612896358927372,
  0.12545388996737336,
  0.1246469092028796,
  0.12426023163147686,
  0.12356388979542765,
  0.12295269906499962,
  0.1226273923480615,
  0.12222259083624057,
  0.12176237304094131,
  0.12128749774011043,
  0.12095361042597834,
  0.12081953075488636,
  0.12048703240264258,
  0.12028838332561054,
  0.12000330292785524,
  0.1196970721964

In [45]:
from sklearn.model_selection import GridSearchCV
### 我们可以创建lgb的sklearn模型，使用上面选择的(学习率，评估器数目)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=20,
                              learning_rate=0.05, n_estimators=437, max_depth=4,
                              metric='mse', bagging_fraction = 0.8,feature_fraction = 0.8)

params_test1={
    'max_depth': range(2,6,1),
    'num_leaves':range(10, 40, 10)
}
gsearch1 = GridSearchCV(estimator=model_lgb, param_grid=params_test1, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=4)
gsearch1.fit(X_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5,
             estimator=LGBMRegressor(bagging_fraction=0.8, feature_fraction=0.8,
                                     learning_rate=0.05, max_depth=6,
                                     metric='mse', n_estimators=437,
                                     num_leaves=50, objective='regression'),
             n_jobs=4,
             param_grid={'max_depth': range(2, 6),
                         'num_leaves': range(10, 40, 10)},
             scoring='neg_mean_squared_error', verbose=1)

In [48]:
mean_squared_error(gsearch1.predict(X_test),y_test)

0.11329909575580044

In [46]:
gsearch1.best_params_#, gsearch1.best_score_

{'max_depth': 4, 'num_leaves': 20}

In [47]:
gsearch1.best_score_

-0.10901238341539951

In [43]:
gsearch1.best_params_#, gsearch1.best_score_

{'max_depth': 4, 'num_leaves': 30}

In [44]:
gsearch1.best_score_

-0.10901238341539951

In [40]:
gsearch1.best_params_#, gsearch1.best_score_

{'max_depth': 5, 'num_leaves': 50}

In [41]:
gsearch1.best_score_

-0.10994185181640681

In [7]:
lgb.feature_importances_# 5 9 11 17 22 28 @70 37 65 38 44 62

array([123, 112, 122, 188,  59,  70,  61,  76,  60,  37, 172,  65,  87,
        59, 103,  53,  83,  38,  82,  77,  90,  69,  44,  54,  81,  92,
        80,  77,  62,  48, 104,  61,  82,  47,  53,  17, 119,  93])

In [12]:
lgb.objective_

'regression'

In [10]:
lgb.n_features_

38

In [14]:
lgb.best_score_

defaultdict(collections.OrderedDict, {})